In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from utils import *
np.random.seed(0)

In [8]:
# READING DATA FROM THE DATASET TO PANDAS DATA FRAME
x=pd.read_excel('DSL-StrongPasswordData.xls')

# REMOVING TWO UNWANTED COLUMN FROM THE DATA SET
del x['sessionIndex']
del x['rep']

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [9]:
# EXTRACTING LIST OF USER NAMES FROM THE DATASET (IN THIS CASE 51)
x1=x.groupby('subject')
user_names=x1.subject.unique()

In [10]:
# CREATING A DICTIONARY CALLED 'dataframes' WHERE DATA IS STORED AS FOLLOWS
# {KEY : VALUE}
# {USER_i : PANDAS DATAFRAME FOR USER i}
# EXAMPLE: {"user1" : data frame for user 1}
user_all_data = {}

# CREATING A DICTIONARY CALLED 'usernames' WHERE ORIGINAL USER NAMES ARE MAPPED TO NEW USER NAMES
# {NEW USER NAME : ORIGINAL USER NAME}
# EXAMPLE: {"user1" : "s002"}
usernames = {}

for i, user in enumerate(user_names):
    
    # CREATING NAME OF USER
    name = "user"+str(i+1)
    
    # STORING USER NAME AND DATA FRAMES TO DICTIONARY
    user_all_data[name] = x1.get_group(user[0])
    usernames[name] = user[0]


In [11]:
# CREATE DATA STRUCTURE FOR TRAIN TEST AND ANAMOLY DATASET
train_data, test_data, anamoly_data = create_data_stucture(user_all_data, usernames)

## EUCLIDEAN

In [12]:
# LOAD TEST TRAIN DATA

metric = 'EUCLIDEAN'

for i,user in enumerate(usernames):
#     user = 'user4'
    train = train_data[user]
    test = test_data[user]
    anamoly = pd.DataFrame()

    # CREATE A LOOP TO CREATE THE ANAMOLY DATASET
    anamoly = create_anamoly_data_set_for_a_user(anamoly_data, user)

    # CREATE MEAN VECTOR
    mean = train.mean(axis = 0) 

    # FUNCTION FOR EUCLEDIAN DISTANCE CALCULATION
    def calculate_dist(train, mean):
        distance = (train - mean).pow(2)
        score = (distance.sum(axis=1)).pow(0.5)
        return score, score.max(), score.min()

    # CALCULATE USER SCORE
    user_score, user_max, user_min = (calculate_dist(test, mean))
    print(user+" test score - " + str(user_score.shape[0]) + " elemnts" )

    # CALCULATE IMPOSTER SCORE
    anamoly_score, anamoly_max, anamoly_min = calculate_dist(anamoly, mean)
    print(user+" impo score - " + str(anamoly_score.shape[0]) + " elemnts" )

    # CREATE SEARCHSPACE FOR OPTIMAL THRESHOLD
    hit_rate_list, miss_rate_list, flase_alarm_rate_list, error, equal_error_index = search_optimal_threshold(user_score, anamoly_score)       

    # PLOT THE ROC CURVE
    plot_ROC(i, user, hit_rate_list, flase_alarm_rate_list, equal_error_index, metric)


TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-12-adbf6fa684f0>, line 34)